<a href="https://colab.research.google.com/github/Enrico-Call/RL-AKI/blob/create-notebook/Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/img/logo_amds.png?raw=1" alt="Logo" width=128px/>

# VUmc Research Project - Reinforcement Learning for Sepsis Prevention
# Data Extraction

AmsterdamUMCdb version 1.0.2 March 2020  
Copyright &copy; 2003-2022 Amsterdam UMC - Amsterdam Medical Data Science

## 1. Set up the environment variables for Colab and GoogleBigQuery to access

In [23]:
import os
import psycopg2
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import io

from google.colab import auth
from google.cloud.bigquery import magics
from google.cloud import bigquery

#sets dateset
DATASET_PROJECT_ID = 'amsterdamumcdb-data'
DATASET_ID = 'ams102'
LOCATION = 'eu'

#sets the project id
PROJECT_ID = "rl-aki"

#all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

#sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID, use_legacy_sql=True)
magics.context.default_query_job_config = def_config

#sets client options job configuration
client_options = {}
client_options['location'] = LOCATION
magics.context.bigquery_client_options = client_options

#BigQuery requires a separate config to prevent the 'BadRequest: 400 Cannot explicitly modify anonymous table' error message
job_config = bigquery.job.QueryJobConfig()

#sets default client settings by re-using the previously defined config
client = bigquery.Client(project=PROJECT_ID, location=LOCATION, default_query_job_config=def_config)

config_gbq = {'query': 
          {'defaultDataset': {
              "datasetId": DATASET_ID, 
              "projectId": DATASET_PROJECT_ID
              },
           'Location': LOCATION}
           }

auth.authenticate_user()
print('Authenticated')

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Authenticated


In [2]:
#get the amsterdamumcdb package from PiPy repository for use in Colab
!pip install amsterdamumcdb
import amsterdamumcdb as adb

     |████████████████████████████████| 199 kB 14.0 MB/s 


### 1.1. Available documentation for AmsterdamUMCdb tables.
Additional information including sample Python code in Jupyter Notebooks and [SQL](https://github.com/AmsterdamUMC/AmsterdamUMCdb/tree/master/amsterdamumcdb/sql) is available from the official [AmsterdamUMCdb GitHub repository](https://github.com/AmsterdamUMC/AmsterdamUMCdb) and [wiki](https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki). 

Table | Wiki | Example code
:- | :--| -:
admissions|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/admissions | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/admissions.ipynb) 
drugitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/drugitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/drugitems.ipynb) 
freetextitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/freetextitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/freetextitems.ipynb)
listitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/lisitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/listitems.ipynb)
numericitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/numericitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/numericitems.ipynb)
procedureorderitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/procedureorderitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/procedureorderitems.ipynb)
processitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/processitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/processitems.ipynb)

**Note**: In contrast to the publicly available version of AmsterdamUMCdb, we have moved *unvalidated* (device) data (`registeredby IS NULL`) from the `numericitems` table to a separate table `unvalidatednumericitems` to reduce query costs when unvalidated device data is not needed. However, depending on the chosen solution, you may need to combine the data from both tables.

## 2. Query AmsterdamUMCdb 

Select variables of interest among patients that were diagnosed with Sepsis during their admission in the ICU at Amsterdam UMC.

Sepsis is a common reason for ICU admission. However, even though all reasons for admission system have a sepsis category, it's often not documented on that category, but instead other conditions were chosen (e.g. sepsis due to intestinal perforation will have the abdominal surgery category selected). To ameliorate this situation, the code in this notebook code uses the following definitions of sepsis:

  -  sepsis_at_admission flagged in the admission form by the attending physician
  -  the admission diagnosis, medical or surgical, is considered a severe infection, e.g., gastro-intestinal perforation, cholangitis, meningitis
  -  use of antibiotics that are not used prophylactically after surgery
  -  use of antibiotics and cultures drawn within 6 hours of admission

To do: sepsis based on Sepsis III-criteria (e.g. qSOFA)

In [87]:
#Loads the admissions table for stratification on admission unit 
admissions = pd.read_gbq(
'''
SELECT * FROM admissions;
'''
, configuration=config_gbq)

#No. patients
no_patients_total = admissions.patientid.nunique()
no_patients_ICU = admissions[admissions['location'].str.contains('IC')].patientid.nunique()
no_patients_MCU = admissions[admissions['location'].str.contains('^MC$')].patientid.nunique()

#No. admissions
no_admissions_total = admissions.admissionid.nunique()
no_admissions_ICU = admissions[admissions['location'].str.contains('IC')].admissionid.nunique()
no_admissions_MCU = admissions[admissions['location'].str.contains('^MC$')].admissionid.nunique()

combined_diagnoses = pd.read_gbq(
    '''
WITH diagnosis_groups AS (
    SELECT admissionid,
        item, 
        CASE
            WHEN itemid IN (
                18669, --NICE APACHEII diagnosen
                18671 --NICE APACHEIV diagnosen
            )
            THEN SPLIT(value,  ' - ')[SAFE_ORDINAL( 1)] 
            -- 'e.g. 'Non-operative cardiovascular - Anaphylaxis' -> Non-operative cardiovascular
            ELSE value
        END as diagnosis_group,
        valueid as diagnosis_group_id,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY 
            CASE --prefer NICE > APACHE IV > II > D
                WHEN itemid = 18671 THEN 6 --NICE APACHEIV diagnosen
                WHEN itemid = 18669 THEN 5 --NICE APACHEII diagnosen                
                WHEN itemid BETWEEN 16998 AND 17017 THEN 4 --APACHE IV diagnosis        
                WHEN itemid BETWEEN 18589 AND 18602 THEN 3 --APACHE II diagnosis
                WHEN itemid BETWEEN 13116 AND 13145 THEN 2 --D diagnosis ICU
                WHEN itemid BETWEEN 16642 AND 16673 THEN 1 --DMC diagnosis Medium Care
            END DESC,
        measuredat DESC) AS rownum
    FROM listitems
    WHERE itemid IN (
        --MAIN GROUP - LEVEL 0
        13110, --D_Hoofdgroep
        16651, --DMC_Hoofdgroep, Medium Care
        
        18588, --Apache II Hoofdgroep
        16997, --APACHE IV Groepen
        
        18669, --NICE APACHEII diagnosen
        18671 --NICE APACHEIV diagnosen
    )
),diagnosis_subgroups AS (
    SELECT admissionid,
        item, 
        value as diagnosis_subgroup,
        valueid as diagnosis_subgroup_id,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY measuredat DESC) AS rownum
    FROM listitems
    WHERE itemid IN (
        --SUB GROUP - LEVEL 1
        13111, --D_Subgroep_Thoraxchirurgie
        16669, --DMC_Subgroep_Thoraxchirurgie
        13112, --D_Subgroep_Algemene chirurgie
        16665, --DMC_Subgroep_Algemene chirurgie
        13113, --D_Subgroep_Neurochirurgie
        16667, --DMC_Subgroep_Neurochirurgie
        13114, --D_Subgroep_Neurologie
        16668, --DMC_Subgroep_Neurologie
        13115, --D_Subgroep_Interne geneeskunde
        16666 --DMC_Subgroep_Interne geneeskunde
    )
), diagnoses AS (
    SELECT admissionid,
        item, 
        CASE
            WHEN itemid IN (
                18669, --NICE APACHEII diagnosen
                18671 --NICE APACHEIV diagnosen
            )
            THEN SPLIT(value,  ' - ')[SAFE_ORDINAL( 2)] 
            -- 'e.g. 'Non-operative cardiovascular - Anaphylaxis' -> Anaphylaxis
            ELSE value
        END as diagnosis,
        CASE
            WHEN itemid IN (
                --SURGICAL
                13116, --D_Thoraxchirurgie_CABG en Klepchirurgie
                16671, --DMC_Thoraxchirurgie_CABG en Klepchirurgie
                13117, --D_Thoraxchirurgie_Cardio anders
                16672, --DMC_Thoraxchirurgie_Cardio anders
                13118, --D_Thoraxchirurgie_Aorta chirurgie
                16670, --DMC_Thoraxchirurgie_Aorta chirurgie
                13119, --D_Thoraxchirurgie_Pulmonale chirurgie
                16673, --DMC_Thoraxchirurgie_Pulmonale chirurgie

                --Not surgical: 13141, --D_Algemene chirurgie_Algemeen   
                --Not surgical: 16642, --DMC_Algemene chirurgie_Algemeen
                13121, --D_Algemene chirurgie_Buikchirurgie
                16643, --DMC_Algemene chirurgie_Buikchirurgie
                13123, --D_Algemene chirurgie_Endocrinologische chirurgie
                16644, --DMC_Algemene chirurgie_Endocrinologische chirurgie
                13145, --D_Algemene chirurgie_KNO/Overige
                16645, --DMC_Algemene chirurgie_KNO/Overige
                13125, --D_Algemene chirurgie_Orthopedische chirurgie
                16646, --DMC_Algemene chirurgie_Orthopedische chirurgie
                13122, --D_Algemene chirurgie_Transplantatie chirurgie
                16647, --DMC_Algemene chirurgie_Transplantatie chirurgie
                13124, --D_Algemene chirurgie_Trauma
                16648, --DMC_Algemene chirurgie_Trauma
                13126, --D_Algemene chirurgie_Urogenitaal
                16649, --DMC_Algemene chirurgie_Urogenitaal
                13120, --D_Algemene chirurgie_Vaatchirurgie
                16650, --DMC_Algemene chirurgie_Vaatchirurgie

                13128, --D_Neurochirurgie _Vasculair chirurgisch
                16661, --DMC_Neurochirurgie _Vasculair chirurgisch
                13129, --D_Neurochirurgie _Tumor chirurgie
                16660, --DMC_Neurochirurgie _Tumor chirurgie
                13130, --D_Neurochirurgie_Overige
                16662, --DMC_Neurochirurgie_Overige

                18596, --Apache II Operatief  Gastr-intenstinaal
                18597, --Apache II Operatief Cardiovasculair
                18598, --Apache II Operatief Hematologisch
                18599, --Apache II Operatief Metabolisme
                18600, --Apache II Operatief Neurologisch
                18601, --Apache II Operatief Renaal
                18602, --Apache II Operatief Respiratoir

                17008, --APACHEIV Post-operative cardiovascular
                17009, --APACHEIV Post-operative gastro-intestinal
                17010, --APACHEIV Post-operative genitourinary
                17011, --APACHEIV Post-operative hematology
                17012, --APACHEIV Post-operative metabolic
                17013, --APACHEIV Post-operative musculoskeletal /skin
                17014, --APACHEIV Post-operative neurologic
                17015, --APACHEIV Post-operative respiratory
                17016, --APACHEIV Post-operative transplant
                17017 --APACHEIV Post-operative trauma

            ) THEN 1
            WHEN itemid = 18669 AND valueid BETWEEN 1 AND 26 THEN 1 --NICE APACHEII diagnosen
            WHEN itemid = 18671 AND valueid BETWEEN 222 AND 452 THEN 1 --NICE APACHEIV diagnosen
            ELSE 0
        END AS surgical,
        valueid as diagnosis_id,
        CASE 
                WHEN itemid = 18671 THEN 'NICE APACHE IV'
                WHEN itemid = 18669 THEN 'NICE APACHE II'
                WHEN itemid BETWEEN 16998 AND 17017 THEN 'APACHE IV'
                WHEN itemid BETWEEN 18589 AND 18602 THEN 'APACHE II'
                WHEN itemid BETWEEN 13116 AND 13145 THEN 'Legacy ICU'
                WHEN itemid BETWEEN 16642 AND 16673 THEN 'Legacy MCU'
        END AS diagnosis_type,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY 
            CASE --prefer NICE > APACHE IV > II > D
                WHEN itemid = 18671 THEN 6 --NICE APACHEIV diagnosen
                WHEN itemid = 18669 THEN 5 --NICE APACHEII diagnosen                
                WHEN itemid BETWEEN 16998 AND 17017 THEN 4 --APACHE IV diagnosis        
                WHEN itemid BETWEEN 18589 AND 18602 THEN 3 --APACHE II diagnosis
                WHEN itemid BETWEEN 13116 AND 13145 THEN 2 --D diagnosis ICU
                WHEN itemid BETWEEN 16642 AND 16673 THEN 1 --DMC diagnosis Medium Care
            END DESC,
            measuredat DESC) AS rownum
    FROM listitems
    WHERE itemid IN (
        -- Diagnosis - LEVEL 2
        --SURGICAL
        13116, --D_Thoraxchirurgie_CABG en Klepchirurgie
        16671, --DMC_Thoraxchirurgie_CABG en Klepchirurgie
        13117, --D_Thoraxchirurgie_Cardio anders
        16672, --DMC_Thoraxchirurgie_Cardio anders
        13118, --D_Thoraxchirurgie_Aorta chirurgie
        16670, --DMC_Thoraxchirurgie_Aorta chirurgie
        13119, --D_Thoraxchirurgie_Pulmonale chirurgie
        16673, --DMC_Thoraxchirurgie_Pulmonale chirurgie
        
        13141, --D_Algemene chirurgie_Algemeen   
        16642, --DMC_Algemene chirurgie_Algemeen
        13121, --D_Algemene chirurgie_Buikchirurgie
        16643, --DMC_Algemene chirurgie_Buikchirurgie
        13123, --D_Algemene chirurgie_Endocrinologische chirurgie
        16644, --DMC_Algemene chirurgie_Endocrinologische chirurgie
        13145, --D_Algemene chirurgie_KNO/Overige
        16645, --DMC_Algemene chirurgie_KNO/Overige
        13125, --D_Algemene chirurgie_Orthopedische chirurgie
        16646, --DMC_Algemene chirurgie_Orthopedische chirurgie
        13122, --D_Algemene chirurgie_Transplantatie chirurgie
        16647, --DMC_Algemene chirurgie_Transplantatie chirurgie
        13124, --D_Algemene chirurgie_Trauma
        16648, --DMC_Algemene chirurgie_Trauma
        13126, --D_Algemene chirurgie_Urogenitaal
        16649, --DMC_Algemene chirurgie_Urogenitaal
        13120, --D_Algemene chirurgie_Vaatchirurgie
        16650, --DMC_Algemene chirurgie_Vaatchirurgie

        13128, --D_Neurochirurgie _Vasculair chirurgisch
        16661, --DMC_Neurochirurgie _Vasculair chirurgisch
        13129, --D_Neurochirurgie _Tumor chirurgie
        16660, --DMC_Neurochirurgie _Tumor chirurgie
        13130, --D_Neurochirurgie_Overige
        16662, --DMC_Neurochirurgie_Overige
        
        18596, --Apache II Operatief  Gastr-intenstinaal
        18597, --Apache II Operatief Cardiovasculair
        18598, --Apache II Operatief Hematologisch
        18599, --Apache II Operatief Metabolisme
        18600, --Apache II Operatief Neurologisch
        18601, --Apache II Operatief Renaal
        18602, --Apache II Operatief Respiratoir
        
        17008, --APACHEIV Post-operative cardiovascular
        17009, --APACHEIV Post-operative gastro-intestinal
        17010, --APACHEIV Post-operative genitourinary
        17011, --APACHEIV Post-operative hematology
        17012, --APACHEIV Post-operative metabolic
        17013, --APACHEIV Post-operative musculoskeletal /skin
        17014, --APACHEIV Post-operative neurologic
        17015, --APACHEIV Post-operative respiratory
        17016, --APACHEIV Post-operative transplant
        17017, --APACHEIV Post-operative trauma

        --MEDICAL
        13133, --D_Interne Geneeskunde_Cardiovasculair
        16653, --DMC_Interne Geneeskunde_Cardiovasculair
        13134, --D_Interne Geneeskunde_Pulmonaal
        16658, --DMC_Interne Geneeskunde_Pulmonaal
        13135, --D_Interne Geneeskunde_Abdominaal
        16652, --DMC_Interne Geneeskunde_Abdominaal
        13136, --D_Interne Geneeskunde_Infectieziekten
        16655, --DMC_Interne Geneeskunde_Infectieziekten
        13137, --D_Interne Geneeskunde_Metabool
        16656, --DMC_Interne Geneeskunde_Metabool
        13138, --D_Interne Geneeskunde_Renaal
        16659, --DMC_Interne Geneeskunde_Renaal
        13139, --D_Interne Geneeskunde_Hematologisch
        16654, --DMC_Interne Geneeskunde_Hematologisch
        13140, --D_Interne Geneeskunde_Overige
        16657, --DMC_Interne Geneeskunde_Overige

        13131, --D_Neurologie_Vasculair neurologisch
        16664, --DMC_Neurologie_Vasculair neurologisch
        13132, --D_Neurologie_Overige
        16663, --DMC_Neurologie_Overige 
        13127, --D_KNO/Overige
        
        18589, --Apache II Non-Operatief Cardiovasculair
        18590, --Apache II Non-Operatief Gastro-intestinaal
        18591, --Apache II Non-Operatief Hematologisch
        18592, --Apache II Non-Operatief Metabolisme
        18593, --Apache II Non-Operatief Neurologisch
        18594, --Apache II Non-Operatief Renaal
        18595, --Apache II Non-Operatief Respiratoir
        
        16998, --APACHE IV Non-operative cardiovascular
        16999, --APACHE IV Non-operative Gastro-intestinal
        17000, --APACHE IV Non-operative genitourinary
        17001, --APACHEIV  Non-operative haematological
        17002, --APACHEIV  Non-operative metabolic
        17003, --APACHEIV Non-operative musculo-skeletal
        17004, --APACHEIV Non-operative neurologic
        17005, --APACHEIV Non-operative respiratory
        17006, --APACHEIV Non-operative transplant
        17007, --APACHEIV Non-operative trauma
        
        --NICE: surgical/medical combined in same parameter
        18669, --NICE APACHEII diagnosen
        18671 --NICE APACHEIV diagnosen
    )
), sepsis AS (
    SELECT
        admissionid,
        CASE valueid
            WHEN 1 THEN 1 --'Ja'
            WHEN 2 THEN 0 --'Nee'
        END as sepsis_at_admission,
        ROW_NUMBER() OVER(
            PARTITION BY 
                admissionid
            ORDER BY 
                measuredat DESC) AS rownum
    FROM listitems
    WHERE 
        itemid = 15808
), sepsis_antibiotics AS ( --non prophylactic antibiotics
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS sepsis_antibiotics_bool,
        STRING_AGG(DISTINCT item, '; ') AS sepsis_antibiotics_given
    FROM drugitems
    WHERE 
        itemid IN (
            6834, --Amikacine (Amukin)
            6847, --Amoxicilline (Clamoxyl/Flemoxin)
            6871, --Benzylpenicilline (Penicilline)
            6917, --Ceftazidim (Fortum)
            --6919, --Cefotaxim (Claforan) -> prophylaxis
            6948, --Ciprofloxacine (Ciproxin)
            6953, --Rifampicine (Rifadin)
            6958, --Clindamycine (Dalacin)
            7044, --Tobramycine (Obracin)
            --7064, --Vancomycine -> prophylaxis for valve surgery
            7123, --Imipenem (Tienam)
            7185, --Doxycycline (Vibramycine)
            --7187, --Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
            --7208, --Erythromycine (Erythrocine) -> often used for gastroparesis
            7227, --Flucloxacilline (Stafoxil/Floxapen)
            7231, --Fluconazol (Diflucan)
            7232, --Ganciclovir (Cymevene)
            7233, --Flucytosine (Ancotil)
            7235, --Gentamicine (Garamycin)
            7243, --Foscarnet trinatrium (Foscavir)
            7450, --Amfotericine B (Fungizone)
            --7504, --X nader te bepalen --non-stock medication
            8127, --Meropenem (Meronem)
            8229, --Myambutol (ethambutol)
            8374, --Kinine dihydrocloride
            --8375, --Immunoglobuline (Nanogam) -> not anbiotic
            --8394, --Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
            8547, --Voriconazol(VFEND)
            --9029, --Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
            9030, --Aztreonam (Azactam)
            9047, --Chlooramfenicol
            --9075, --Fusidinezuur (Fucidin) -> prophylaxis
            9128, --Piperacilline (Pipcil)
            9133, --Ceftriaxon (Rocephin)
            --9151, --Cefuroxim (Zinacef) -> often used for GI/transplant surgical prophylaxis
            --9152, --Cefazoline (Kefzol) -> prophylaxis for cardiac surgery
            9458, --Caspofungine
            9542, --Itraconazol (Trisporal)
            --9602, --Tetanusimmunoglobuline -> prophylaxis/not antibiotic
            12398, --Levofloxacine (Tavanic)
            12772, --Amfotericine B lipidencomplex  (Abelcet)
            15739, --Ecalta (Anidulafungine)
            16367, --Research Anidulafungin/placebo
            16368, --Research Caspofungin/placebo
            18675, --Amfotericine B in liposomen (Ambisome )
            19137, --Linezolid (Zyvoxid)
            19764, --Tigecycline (Tygacil)
            19773, --Daptomycine (Cubicin)
            20175 --Colistine
        )
        AND start < 24*60*60*1000 --within 24 hours (to correct for antibiotics administered before ICU)
    GROUP BY admissionid
), other_antibiotics AS ( --'prophylactic' antibiotics that may be used for sepsis
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS other_antibiotics_bool,
        STRING_AGG(DISTINCT item, '; ') AS other_antibiotics_given
    FROM drugitems
    WHERE 
        itemid IN (
            7064, --Vancomycine -> prophylaxis for valve surgery
            7187, --Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
            8394, --Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
            9029, --Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
            9151, --Cefuroxim (Zinacef) -> often used for GI surgical prophylaxis
            9152 --Cefazoline (Kefzol) -> prophylaxis
        )
        AND start < 24*60*60*1000 --within 24 hours (to correct for antibiotics administered before ICU)
    GROUP BY admissionid    
), cultures AS (
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS sepsis_cultures_bool,
        STRING_AGG(DISTINCT item, '; ') AS sepsis_cultures_drawn
    FROM procedureorderitems
    WHERE 
        itemid IN (
        --8097, --Sputumkweek afnemen -> often used routinely
        --8418, --Urinekweek afnemen
        --8588, --MRSA kweken afnemen 
        9189, --Bloedkweken afnemen
        9190, --Cathetertipkweek afnemen
        --9191, --Drainvochtkweek afnemen
        --9192, --Faeceskweek afnemen -> Clostridium
        --9193, --X-Kweek nader te bepalen
        --9194, --Liquorkweek afnemen
        --9195, --Neuskweek afnemen
        --9197, --Perineumkweek afnemen -> often used routinely
        -9198, --Rectumkweek afnemen -> often used routinely
        9200, --Wondkweek afnemen
        9202, --Ascitesvochtkweek afnemen
        --9203, --Keelkweek afnemen -> often used routinely
        --9204, --SDD-kweken afnemen -> often used routinely
        9205 --Legionella sneltest (urine)
        --1302, --SDD Inventarisatiekweken afnemen -> often used routinely
        --19663, --Research Neuskweek COUrSe
        --19664, --Research Sputumkweek COUrSe
        )
        AND registeredat < 6*60*60*1000 --within 6 hours
    GROUP BY admissionid
)
SELECT 
    admissions.* 
    , diagnosis_type
    , diagnosis, diagnosis_id
    , diagnosis_subgroup
    , diagnosis_subgroup_id
    , diagnosis_group
    , diagnosis_group_id
    , surgical
    , sepsis_at_admission
    , sepsis_antibiotics_bool
    , sepsis_antibiotics_given
    , other_antibiotics_bool
    , other_antibiotics_given
    , sepsis_cultures_bool
    , sepsis_cultures_drawn
FROM admissions
LEFT JOIN diagnoses on admissions.admissionid = diagnoses.admissionid
LEFT JOIN diagnosis_subgroups on admissions.admissionid = diagnosis_subgroups.admissionid
LEFT JOIN diagnosis_groups on admissions.admissionid = diagnosis_groups.admissionid
LEFT JOIN sepsis on admissions.admissionid = sepsis.admissionid
LEFT JOIN sepsis_antibiotics on admissions.admissionid = sepsis_antibiotics.admissionid
LEFT JOIN other_antibiotics on admissions.admissionid = other_antibiotics.admissionid
LEFT JOIN cultures on admissions.admissionid = cultures.admissionid
WHERE --only last updated record
    (diagnoses.rownum = 1 OR diagnoses.rownum IS NULL) AND 
    (diagnosis_subgroups.rownum = 1 OR diagnosis_subgroups.rownum IS NULL) AND
    (diagnosis_groups.rownum = 1 OR diagnosis_groups.rownum IS NULL) AND
    (sepsis.rownum = 1 OR sepsis.rownum IS NULL) 
;
    '''
    , configuration=config_gbq)

#surgical
surgical_total = len(combined_diagnoses[combined_diagnoses['surgical'] == 1])
surgical_pct_total = 100*surgical_total/no_admissions_total

surgical_ICU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 1)  & combined_diagnoses['location'].str.contains('IC')])
surgical_pct_ICU = 100*surgical_ICU/no_admissions_ICU

surgical_MCU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 1)  & combined_diagnoses['location'].str.contains('^MC$')])
surgical_pct_MCU = 100*surgical_MCU/no_admissions_MCU

#medical
medical_total = len(combined_diagnoses[combined_diagnoses['surgical'] == 0])
medical_pct_total = 100*medical_total/no_admissions_total

medical_ICU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 0)  & combined_diagnoses['location'].str.contains('IC')])
medical_pct_ICU = 100*medical_ICU/no_admissions_ICU

medical_MCU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 0)  & combined_diagnoses['location'].str.contains('^MC$')])
medical_pct_MCU = 100*medical_MCU/no_admissions_MCU

#surgical/medical unknown
medical_surgical_na_total = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical'])])
medical_surgical_na_pct_total = 100*medical_surgical_na_total/no_admissions_total

medical_surgical_na_ICU = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical']) & combined_diagnoses['location'].str.contains('IC')])
medical_surgical_na_pct_ICU = 100*medical_surgical_na_ICU/no_admissions_ICU

medical_surgical_na_MCU = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical']) & combined_diagnoses['location'].str.contains('^MC$')])
medical_surgical_na_pct_MCU = 100*medical_surgical_na_MCU/no_admissions_MCU

#urgency
urgency_total = len(combined_diagnoses[combined_diagnoses['urgency'] == '1']) #apparently the postgresql bit become an object
urgency_pct_total = 100*urgency_total/no_admissions_total

urgency_ICU = len(combined_diagnoses[(combined_diagnoses['urgency'] == '1')  & combined_diagnoses['location'].str.contains('IC')])
urgency_pct_ICU = 100*urgency_ICU/no_admissions_ICU

urgency_MCU = len(combined_diagnoses[(combined_diagnoses['urgency'] == '1')  & combined_diagnoses['location'].str.contains('^MC$')])
urgency_pct_MCU = 100*urgency_MCU/no_admissions_MCU

#Surgical

re_cardiosurg = r'(CABG|AVR|hartchirurgie|heart surgery|Chron. cardiovasculaire ziekte|hartkleppen' + \
r'|cardiovascula|MVP|MVR|mitral|tricuspid|pericard|aortic.*valve|lobectom|segment|thorax|Bentall|aorta-ascendens|aorta-boog' + \
r'|aorta-wortel|aorta-descendens|lung|pneumectomie|bullectom|respiratoir neoplasm|thoracoscop|thoracotom(y|ie)|respirato' + \
r'|vrije wand ruptuur|VSR|ASD|pleurectom|intracardiac|aneurysmectom|congenital defect repair)(?! for esophag)'
re_neurosurg = r'neuro|tentorieel|cranial|subdur|epidur|subarachn|intracerbr|hoofdtrauma|SAB|S.A.H.|CNS|Hoofd|seizures' + \
r'|craniotom|cranioplast|spinal|dwarslaesie|ventriculstom|transphenoidal|brain|A.V.M.|Arteriovenous malformation' + \
r'|brughoek|shunts and revisions|stereotactic|Burr hole|cerebrospinal'
re_vascsurg = r'vaatchirurgie|vasc.*surg|EVAR|aorta vervanging|perifeer vasc|embolectom|aneurysm|carotid|endovasc' + \
r'|dissectie|endarterectom|thrombectomy|dilatation|PTCA|all other bypass|femoral-popliteal|aorto-femoral|femoral-femoral' + \
r'|transplantectom'
re_gisurg = r'oesophagus|esophageal|maag|stomach|darm|intestin|gastro-intestin|pancreatitis|laparotom|gastro-intestinale perforatie' + \
r'|galblaas|Bleeding-.*GI|other GI|colon|rectal|GI.*surgery|GI obstruction|Whipple|diverticular|appendectomy' + \
r'|peritonitis|cholecystectomy|exenteration'
re_uro = r'(?<!ne)(urolog|cystectomy|genitourinary surgery|prostatectom|ileal\-conduit|orchiectomy|bladder repair' + \
r'|nefrectom|nephrectom|renaal neopsplama)'
re_obgyn = r'hysterectom|Cesarean|ectopic pregnancy'
re_gensurg = r'mond/keel|cancer oral|cancer laryngeal|spondylodes|Fusion-spinal|devices for spine|orthopedic' + \
r'|renaal|metabol|endocrin|thyroid|hip replacement|knee replacement|adrenalectom|tracheostomy|bewaking|amputation|apnea-sleep' + \
r'|lymph node dissection|mastectomy|cosmetic|fracture-pathological|bewaking'
re_trauma_surg = r'(?<!non-)(?<!see )(trauma|hypotherm|smoke inhalation)(?!, see trauma)(?! see)(?!: see)'
re_tx_surg = r'niertransplantatie|kidney transplant|renaal transplantatie|pancreastransplantatie'

re_respfailure_surg = r'resp.*insuff|na respiratoir arrest|arrest, respiratory|atelectas|pneumoni|na ok'

re_sepsis_surg = r'sepsis|pneumoni|GI perforation|perforation/rupture|infection|abscess|GI Vascular ischemia|diverticular|appendectomy|peritonitis'
re_cardiacarrest_surg = r'Cardiac arrest.*respiratory arrest|na reanimatie'
re_hepa = r'lever'
re_surg_other = r'diagnose anders|respiratoir|cardiovasculair|niet operatief'

re_surgical_medical = r'Bloeding tractus digestivus|Haemorragische shock|Gastro-intestinale bloeding|Bleeding, upper GI' + \
r'|hematologisch|Hematologische maligniteit|Haematologisch|Renaal'

re_surgical = '(' + \
    re_cardiosurg + '|' + \
    re_neurosurg + '|' + \
    re_vascsurg + '|' + \
    re_gisurg + '|' + \
    re_uro + '|' + \
    re_obgyn + '|' + \
    re_gensurg + '|' + \
    re_trauma_surg + '|' + \
    re_tx_surg + '|' + \
    re_hepa + '|' + \
    re_surg_other + '|' + \
    re_surgical_medical + '|' + \
    re_respfailure_surg + '|' + \
    re_sepsis_surg + '|' + \
    re_cardiacarrest_surg + '|' + \
    ')'

#check to see if we caught all (should return empty data frame):
combined_diagnoses[(combined_diagnoses['surgical'] == 1
) & ~(combined_diagnoses['diagnosis'].str.contains(
        re_surgical, na=False, flags=re.IGNORECASE))].groupby(['diagnosis_group','diagnosis']).size().to_frame('number of admissions').sort_values('number of admissions', ascending=False).reset_index()

#Medical

re_respfailure_med = r'(?<! without )(resp.*insuff|pneumoni|respirato|luchtweg obstructie|obstruction-airway' + \
r'|chronisch obstructieve longziekte|emphysema|asthma|aspiratie|aspiration|longembolie|pulmonary|pulmonaire' + \
r'|long|lung|atelectas|ALI|ARDS|pleural|cancer, lung|pneumothorax|verdrinking|near drowning|weaning|hemothorax|PCP)'
re_cardiacarrest_med = r'Cardiac arrest.*respiratory arrest|na reanimatie'
re_sepsis_med = r'sepsis|septic|infect|pneumoni|cholangitis|pancr|endocarditis|meningitis|GI perforation|abces|abscess|darm ischaemie|GI vascular|fasciitis' + \
r'|inflammatory|peritonitis'
re_cardio = r'cardiogene shock|shock, cardiogenic|angina|ritme|rhythm|cardiovascular|cardiovasculair' + \
r'|myocardial|endocarditis|coronair|coronary|cardiomyopath|tamponade|pericardial|papillary muscle|^MI|hartkleppen' + \
r'|CHF|hart falen|decompensatio cordis'
re_neuro = '(?<!see )(insult|seizure|CVA|observatie neurologische status|intracerebraal haematoom|intracranial|intracerebr|subdur|subarachno' + \
r'|epidur|coma|neurologisch|neurologic|CZS|S\.A\.B\.|neurologie|hoofdtrauma|head|neuro|muscula|spinal|meningitis|encephalitis' + \
r'|myasthenia|vaatspasme protocol|Guillian-Barre|encephalopath|musculoskeletal)(?!-see Neurological System)'
re_bleeding = r'bloeding tractus digestivus|gastro-intestinale bloeding|gastro-intestinaal|bleeding, upper GI|bleeding, lower GI' + \
r'|bleeding, GI|ulcer'
re_gi_med = r'ileus|GI medical|GI obstruction'
re_tox = r'intox|overdosis|overdose|toxicity|withdrawal|drug'
re_trauma_med = r'(?<!non-)(?<!see )(trauma|hypotherm|smoke inhalation)(?!, see trauma)(?! see)(?!: see)'
re_hemonc = r'malign|hematolog|cancer|bloeding|TTP|HUS|leukemi|pancytopen|coagulopath|anemia|neutropen|lymph|sickle'
re_endo_med = r'metabolisme|keto-acidose|diabetic|metabolic|endocrine|hypertens|acid-base|hypoglyc|thyroid'
re_shock_med = r'hypovolemische shock|shock|haemorr|hemorrha|anaphylaxis'
re_nefro_med = r'renaal|renal|tubulus|genitourinary|urolog|nefr'
re_hepa_med = r'lever|hepatic'
re_obgyn_med = r'obstetrie|postpartum|eclampsia'
re_vasc_med = r'aneurysm|vascular|cardiovascular medical|Thrombus,arterial|vascular medical|dissect|EVAR|embolectom'
re_mon_med = r'apnea, sleep|monitoring|bewaking|observatie'
re_tx_med = r'transplant'

re_medical = '(' + \
re_respfailure_med + '|' + \
re_cardiacarrest_med + '|' + \
re_sepsis_med + '|' + \
re_cardio + '|' + \
re_neuro + '|' + \
re_bleeding + '|' + \
re_gi_med + '|' + \
re_tox + '|' + \
re_trauma_med + '|' + \
re_hemonc + '|' + \
re_endo_med + '|' + \
re_shock_med + '|' + \
re_nefro_med + '|' + \
re_hepa_med + '|' + \
re_obgyn_med + '|' + \
re_vasc_med + '|' + \
re_mon_med + '|' + \
re_tx_med + '|' + \
')'

#check to see if we caught all (should return empty data frame):
combined_diagnoses[(combined_diagnoses['surgical'] == 0
) & ~(combined_diagnoses['diagnosis'].str.contains(
        re_medical, na=False, flags=re.IGNORECASE))].groupby(['diagnosis_group','diagnosis']).size().to_frame('number of admissions').sort_values('number of admissions', ascending=False).reset_index()

#Extract patients that had a diagnosis of Sepsis

sepsis = combined_diagnoses[
    (
        (
            #use reasons for admission
            #surgical admissions with sepsis
            (combined_diagnoses['surgical'] == 1) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_surg, na=False, flags=re.IGNORECASE))
        ) | (
            #medical admissions with sepsis
            (combined_diagnoses['surgical'] == 0) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_med, na=False, flags=re.IGNORECASE))
        ) | (
            #uses documentation at admission form (Early Goal Directed Therapy)
            (combined_diagnoses['sepsis_at_admission'] == 1)
        ) | (
            #uses administered (therapeutic) antibiotics for determining sepsis
            (combined_diagnoses['sepsis_antibiotics_bool'] == 1)
        ) | (
            #uses combination of administered antibiotics (that sometimes are used as prophylaxis) AND 
            #drawn cultures for determining sepsis
            (combined_diagnoses['other_antibiotics_bool'] == 1) 
            & (combined_diagnoses['sepsis_cultures_bool'] == 1)
        )
    ) & ~(combined_diagnoses['sepsis_at_admission'] == 0) 
    #exclude all diagnoses where explicitly 'no sepsis' was documented
]
sepsis_counts = sepsis.fillna('N/A').groupby(['diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
sepsis_counts['n_admissions'].sum()

# sepsis.reset_index(inplace=True)
ids = sepsis['admissionid'].to_string(index=False)
ids = ids.replace('\n', ',')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:508: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:536: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [93]:
state_space_query = pd.read_gbq(
    '''
SELECT
    admissionid,
    itemid,
    item,
    value,
    registeredby,
    CASE
        WHEN registeredby NOT LIKE '%Systeem%' THEN TRUE
        ELSE FALSE
    END AS manual_entry,
FROM numericitems 
WHERE itemid IN (
        6840,   --Natrium
        6642,   --ABP Mean Blood Pressure
        6679,   --Niet invasieve bloeddruk gemiddeld
        6835,   --Kalium mmol/l
        6836,   --Kreatinine µmol/l (erroneously documented as µmol)
        8794,   --UrineCAD
        8796,   --UrineSupraPubis
        8798,   --UrineSpontaan
        8800,   --UrineIncontinentie
        8803,   --UrineUP
        8843,   --ABP gemiddeld II
        9924,   --Natrium (bloed)
        9927,   --Kalium (bloed) mmol/l
        9941,   --Kreatinine (bloed) µmol/l
        9555,   --Natrium Astrup
        9556,   --Kalium Astrup mmol/l
        10284,  --Na (onv.ISE) (bloed)
        10285,  --K (onv.ISE) (bloed) mmol/l
        10743,  --Nefrodrain li Uit
        10745,  --Nefrodrain re Uit
        14216,  --KREAT enzym. (bloed) µmol/l
        19921,  --UrineSplint Li
        19922   --UrineSplint Re
    )
    AND
    (admissionid IN (
    '''+
    ids+
    '''
    ))
    '''
    , configuration=config_gbq)

In [95]:
len(state_space_query['admissionid'].unique())

3132

In [11]:
state_space_query = pd.read_gbq(
    '''
    WITH baseline AS (
    SELECT n.admissionid,
    MIN(n.value) AS baseline_creatinine
    FROM numericitems n
    LEFT JOIN admissions a ON
        n.admissionid = a.admissionid
    WHERE itemid IN (
        6840, --Natrium
        6642,  --ABP Mean Blood Pressure
        6679, --Niet invasieve bloeddruk gemiddeld
        6835, --Kalium mmol/l
        6836, --Kreatinine µmol/l (erroneously documented as µmol)
        8794,  --UrineCAD
        8796,  --UrineSupraPubis
        8798, --UrineSpontaan
        8800, --UrineIncontinentie
        8803, --UrineUP
        8843, --ABP gemiddeld II
        9924, --Natrium (bloed)
        9927, --Kalium (bloed) mmol/l
        9941, --Kreatinine (bloed) µmol/l
        9555, --Natrium Astrup
        9556, --Kalium Astrup mmol/l
        10284, --Na (onv.ISE) (bloed)
        10285, --K (onv.ISE) (bloed) mmol/l
        10743,--Nefrodrain li Uit
        10745, --Nefrodrain re Uit
        14216, --KREAT enzym. (bloed) µmol/l
        19921, --UrineSplint Li
        19922 --UrineSplint Re
    ) AND
    --search 1 year befor admission
    (n.measuredat - a.admittedat)/(60*60*1000) > -(365*24) AND (n.measuredat - a.admittedat) < (24*60*60*1000)
    GROUP BY n.admissionid
),
max_creat AS (
    SELECT n.admissionid,
    MAX(n.value) AS max_creatinine_7days
    FROM numericitems n
    LEFT JOIN admissions a ON
        n.admissionid = a.admissionid
    WHERE itemid IN (
        6840, --Natrium
        6642,  --ABP Mean Blood Pressure
        6679, --Niet invasieve bloeddruk gemiddeld
        6835, --Kalium mmol/l
        6836, --Kreatinine µmol/l (erroneously documented as µmol)
        8794,  --UrineCAD
        8796,  --UrineSupraPubis
        8798, --UrineSpontaan
        8800, --UrineIncontinentie
        8803, --UrineUP
        8843, --ABP gemiddeld II
        9924, --Natrium (bloed)
        9927, --Kalium (bloed) mmol/l
        9941, --Kreatinine (bloed) µmol/l
        9555, --Natrium Astrup
        9556, --Kalium Astrup mmol/l
        10284, --Na (onv.ISE) (bloed)
        10285, --K (onv.ISE) (bloed) mmol/l
        10743,--Nefrodrain li Uit
        10745, --Nefrodrain re Uit
        14216, --KREAT enzym. (bloed) µmol/l
        19921, --UrineSplint Li
        19922 --UrineSplint Re
    ) AND
    --within 7 days of admission
    (n.measuredat - a.admittedat) > 0 AND (n.measuredat - a.admittedat) < (7*24*60*60*1000)
    GROUP BY n.admissionid
)
SELECT
    n.admissionid,
    n.itemid,
    n.item,
    n.value,
    n.registeredby,
    CASE
        WHEN n.registeredby NOT LIKE '%Systeem%' THEN TRUE
        ELSE FALSE
    END AS manual_entry,
    (n.measuredat - a.admittedat)/(1000*60) AS time,
    b.baseline_creatinine,
    m.max_creatinine_7days,
    CASE
        -- AKI definition: 3 fold increase:
        WHEN baseline_creatinine > 0 AND m.max_creatinine_7days/baseline_creatinine > 3 THEN TRUE
        -- AKI definition: increase to >= 354 umol/l AND at least 44 umol/l increase:
        WHEN max_creatinine_7days >= 354 AND max_creatinine_7days - baseline_creatinine >= 44 THEN TRUE
        ELSE FALSE
    END AS acute_renal_failure
FROM numericitems n
LEFT JOIN admissions a ON
    n.admissionid = a.admissionid
LEFT JOIN baseline b ON -- get the baseline kreat (before admission)
    n.admissionid = b.admissionid
LEFT JOIN max_creat m ON --get the highest within a week of admission
    n.admissionid = m.admissionid
WHERE n.itemid IN (
    6840, --Natrium
        6642,  --ABP Mean Blood Pressure
        6679, --Niet invasieve bloeddruk gemiddeld
        6835, --Kalium mmol/l
        6836, --Kreatinine µmol/l (erroneously documented as µmol)
        8794,  --UrineCAD
        8796,  --UrineSupraPubis
        8798, --UrineSpontaan
        8800, --UrineIncontinentie
        8803, --UrineUP
        8843, --ABP gemiddeld II
        9924, --Natrium (bloed)
        9927, --Kalium (bloed) mmol/l
        9941, --Kreatinine (bloed) µmol/l
        9555, --Natrium Astrup
        9556, --Kalium Astrup mmol/l
        10284, --Na (onv.ISE) (bloed)
        10285, --K (onv.ISE) (bloed) mmol/l
        10743,--Nefrodrain li Uit
        10745, --Nefrodrain re Uit
        14216, --KREAT enzym. (bloed) µmol/l
        19921, --UrineSplint Li
        19922 --UrineSplint Re
    )
-- measurements within 24 hours of ICU stay (use 30 minutes before admission to allow for time differences):
AND (n.measuredat - a.admittedat) <= 1000*60*60*24 AND (n.measuredat - a.admittedat) >= -(1000*60*30) 
    '''
    , configuration=config_gbq)

In [16]:
reasons_for_admission

,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,...,diagnosis_group,diagnosis_group_id,surgical,sepsis_at_admission,sepsis_antibiotics_bool,sepsis_antibiotics_given,other_antibiotics_bool,other_antibiotics_given,sepsis_cultures_bool,sepsis_cultures_drawn
0,13118,15198,1,IC,1,Eerste Hulp afdeling ander ziekenhuis,0,2010-2016,33900000,9,...,Non-operative genitourinary,76.0,0.0,0.0,NaN,None,NaN,None,NaN,None
1,18318,21215,2,MC,0,Recovery zelfde ziekenhuis (alleen bij niet ge...,6269460000,2010-2016,6357660000,25,...,Post-operative musculoskeletal /skin,359.0,1.0,0.0,NaN,None,NaN,None,NaN,None
2,5803,6720,4,IC,0,CCU/IC ander ziekenhuis,88133040000,2010-2016,88666080000,148,...,Non-operative genitourinary,77.0,0.0,1.0,NaN,None,NaN,None,NaN,None
3,19175,22195,1,MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,72720000,21,...,Post-operative genitourinary,325.0,1.0,0.0,NaN,None,NaN,None,NaN,None
4,1046,1208,1,IC,0,CCU/IC ander ziekenhuis,0,2010-2016,361500000,101,...,Non-operative genitourinary,76.0,0.0,0.0,NaN,None,NaN,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23101,11158,12929,1,IC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,2615340000,726,...,Post-operative gastro-intestinal,291.0,1.0,0.0,1.0,Ceftriaxon (Rocephin),1.0,Metronidazol (Flagyl),NaN,None
23102,13190,15280,1,IC,0,"Andere locatie zelfde ziekenhuis, transport pe...",0,2010-2016,211140000,59,...,Post-operative gastro-intestinal,304.0,1.0,0.0,1.0,Amoxicilline (Clamoxyl/Flemoxin); Ceftazidim (...,1.0,Metronidazol (Flagyl),NaN,None
23103,14177,16426,1,IC,1,Recovery zelfde ziekenhuis (alleen bij niet ge...,0,2010-2016,1057140000,293,...,Post-operative gastro-intestinal,305.0,1.0,0.0,1.0,Ceftriaxon (Rocephin),1.0,Metronidazol (Flagyl),NaN,None
23104,14551,16855,1,IC,1,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,147120000,40,...,Post-operative gastro-intestinal,305.0,1.0,1.0,1.0,Gentamicine (Garamycin); Ceftriaxon (Rocephin),1.0,Metronidazol (Flagyl),NaN,None
